#**Adding Demographic data to Raw PPG Signals**

---
##**Importing Libraries**

In [1]:
import pandas as pd
import numpy as np


##**Load Datasets**

In [2]:
# PPG dataset
ppg_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/ds/Master-ppg-data.csv")

# Clinical tables
patients_df   = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/ALL DATA SETS/matched/PATIENTS.csv")
admissions_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/ALL DATA SETS/matched/ADMISSIONS.csv")
diagnoses_df  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/ALL DATA SETS/matched/DIAGNOSES_ICD.csv")


#**Derive SUBJECT_ID from patient_id**

In [3]:
# patient_id format: p010023 → SUBJECT_ID = 10023
ppg_df["SUBJECT_ID"] = ppg_df["patient_id"].str[1:].astype(int)


##**Attach gender and DOB**

In [4]:
ppg_df = ppg_df.merge(
    patients_df[["SUBJECT_ID", "GENDER", "DOB"]],
    on="SUBJECT_ID",
    how="left"
)


##**Compute age (standard MIMIC method)**

In [5]:
# Datetime safety
patients_df["DOB"] = pd.to_datetime(patients_df["DOB"], errors="coerce")
admissions_df["ADMITTIME"] = pd.to_datetime(admissions_df["ADMITTIME"], errors="coerce")

# First admission per patient
first_admit = (
    admissions_df
    .sort_values("ADMITTIME")
    .groupby("SUBJECT_ID", as_index=False)
    .first()[["SUBJECT_ID", "ADMITTIME"]]
)

# Merge DOB
age_df = first_admit.merge(
    patients_df[["SUBJECT_ID", "DOB"]],
    on="SUBJECT_ID",
    how="left"
)

# Compute age using YEAR difference (MIMIC standard)
age_df["AGE"] = age_df["ADMITTIME"].dt.year - age_df["DOB"].dt.year

# Clean impossible values
age_df.loc[age_df["AGE"] < 0, "AGE"] = np.nan
age_df["AGE"] = age_df["AGE"].clip(upper=90)

# Merge AGE
ppg_df = ppg_df.merge(
    age_df[["SUBJECT_ID", "AGE"]],
    on="SUBJECT_ID",
    how="left"
)


In [6]:
age_cols = [c for c in ppg_df.columns if c.startswith("AGE")]

if "AGE_y" in age_cols:
    ppg_df["AGE"] = ppg_df["AGE_y"]
    ppg_df = ppg_df.drop(columns=age_cols)


##**Aggregate ICD-9 codes per patient**

In [7]:
diag_sub = diagnoses_df[["SUBJECT_ID", "ICD9_CODE"]].dropna()

icd_agg = (
    diag_sub
    .groupby("SUBJECT_ID")["ICD9_CODE"]
    .apply(lambda x: sorted(set(x)))
    .reset_index()
)

icd_agg["ICD9_CODES"] = icd_agg["ICD9_CODE"].apply(lambda x: ";".join(x))
icd_agg = icd_agg[["SUBJECT_ID", "ICD9_CODES"]]

ppg_df = ppg_df.merge(
    icd_agg,
    on="SUBJECT_ID",
    how="left"
)


In [8]:
print(ppg_df.columns.tolist())


['patient_id', 'group', 'record_id', 'fs', 'ppg_min_3', 'ppg_min_4', 'ppg_min_5', 'ppg_min_6', 'ppg_min_7', 'ppg_min_8', 'SUBJECT_ID', 'GENDER', 'DOB', 'AGE', 'ICD9_CODES']


##**Final column order & save**

In [9]:
final_cols = [
    "patient_id",
    "SUBJECT_ID",
    "group",
    "record_id",
    "fs",
    "AGE",
    "GENDER",
    "ICD9_CODES",
    "ppg_min_3",
    "ppg_min_4",
    "ppg_min_5",
    "ppg_min_6",
    "ppg_min_7",
    "ppg_min_8"
]

# Safety check
missing = [c for c in final_cols if c not in ppg_df.columns]
print("Missing columns:", missing)

ppg_master = ppg_df[final_cols]


Missing columns: []


In [10]:
import os
output_path = "/content/drive/MyDrive/Colab Notebooks/Final Project/ds/master-demographic-ppg-data.csv"

os.makedirs(os.path.dirname(output_path), exist_ok=True)

ppg_master.to_csv(output_path, index=False)

print(f"Saved master dataset with {len(ppg_master)} patients")
print(f"File saved at: {output_path}")


Saved master dataset with 5147 patients
File saved at: /content/drive/MyDrive/Colab Notebooks/Final Project/ds/master-demographic-ppg-data.csv


In [11]:
ppg_master.tail()

,patient_id,SUBJECT_ID,group,record_id,fs,AGE,GENDER,ICD9_CODES,ppg_min_3,ppg_min_4,ppg_min_5,ppg_min_6,ppg_min_7,ppg_min_8
5142,p099946,99946,p09,3382559_0012,125,50.0,M,0416;2724;2763;30000;3019;3441;43811;43813;438...,0.5294 0.5216 0.5176 0.5059 0.4980 0.4902 0.47...,0.7255 0.7137 0.7059 0.6941 0.6784 0.6667 0.64...,0.6392 0.6275 0.6157 0.6039 0.5922 0.5804 0.56...,0.7098 0.7176 0.7216 0.7216 0.7176 0.7098 0.70...,0.4745 0.5216 0.5608 0.5882 0.6157 0.6392 0.65...,0.6431 0.6275 0.6157 0.6000 0.5804 0.5647 0.54...
5143,p099955,99955,p09,3064062_0001,125,63.0,M,25060;2760;3484;3485;3572;4019;4321;78003;9965...,1.4819 1.4682 1.4545 1.4399 1.4252 1.4096 1.39...,2.0156 2.0186 2.0215 2.0225 2.0225 2.0205 2.01...,2.4741 2.4702 2.4653 2.4585 2.4506 2.4399 2.42...,1.1701 1.1525 1.1339 1.1153 1.0987 1.0831 1.06...,1.9570 1.9286 1.9022 1.8788 1.8622 1.8524 1.84...,1.5963 1.5806 1.5679 1.5562 1.5435 1.5318 1.52...
5144,p099982,99982,p09,3917821_0009,125,65.0,M,2763;2768;2875;3970;4168;4254;42731;4280;42823...,0.4555 0.4575 0.4594 0.4614 0.4643 0.4673 0.47...,0.4868 0.4917 0.4976 0.5034 0.5093 0.5152 0.52...,0.4203 0.4203 0.4194 0.4184 0.4164 0.4154 0.41...,0.6804 0.6804 0.6804 0.6794 0.6784 0.6784 0.67...,0.5230 0.5230 0.5239 0.5249 0.5259 0.5259 0.52...,0.5073 0.5083 0.5083 0.5093 0.5103 0.5112 0.51...
5145,p099983,99983,p09,3757366_0004,125,79.0,M,2724;4019;41001;41401;5849;60001;78820;V4582,0.4399 0.4330 0.4252 0.4174 0.4106 0.4037 0.39...,0.6422 0.6540 0.6647 0.6735 0.6804 0.6872 0.69...,0.3363 0.3353 0.3353 0.3363 0.3372 0.3382 0.34...,0.3050 0.2952 0.2854 0.2766 0.2678 0.2620 0.25...,0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.00...,0.5005 0.5005 0.5005 0.5005 0.5005 0.5005 0.50...
5146,p099992,99992,p09,3301572_0037,125,66.0,F,2721;2851;3004;4019;53081;56881;5699;5772;5799...,0.6196 0.6078 0.5961 0.5843 0.5725 0.5608 0.54...,0.3529 0.3922 0.4392 0.4902 0.5412 0.5882 0.63...,0.2510 0.2510 0.2510 0.2510 0.2510 0.2510 0.25...,0.2824 0.2824 0.2784 0.2745 0.2706 0.2706 0.26...,0.3294 0.3294 0.3255 0.3216 0.3216 0.3176 0.31...,0.6118 0.6000 0.5843 0.5686 0.5529 0.5373 0.51...
